In [28]:
import urllib
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

'''
    ['11', 'Тюнинг', 'Стайлинг-Тюнинг-Доп-оборудование-Защита.24', 'FW POLO'],
    ['12', 'Эксплуатация-Ремонт', 'Ремонт-и-доработки-своими-силами.48', 'FW POLO'],
    ['13', 'Эксплуатация-Вопросы', 'Вопросы-возникшие-при-эксплуатации-автомобиля.47', 'FW POLO'],
    ['14', 'Проблемы-Заводской брак', 'Проблемы-заводской-брак-и-тд.44', 'FW POLO']
'''

site = 'http://www.polosedan-club.com/'
folder_names = [
    ['0', 'Отзывы', 'Отзывы-владельцев-о-Поло-седан-polo-sedan.13', 'FW POLO'],
    ['1', 'Двигатель', 'Двигатель-и-его-системы.18', 'FW POLO'],
    ['2', 'Трансмиссия', 'Сцепление-и-коробка-передач.19', 'FW POLO'],
    ['3', 'Ходовая', 'Ходовая-и-подвеска.20', 'FW POLO'],
    ['4', 'Колеса', 'Шины-и-колесные-диски.32', 'FW POLO'],
    ['5', 'Электрооборудование', 'Электрооборудование.21', 'FW POLO'],
    ['6', 'Мультимедиа', 'Аудио-Видео-системы-автомобиля.43', 'FW POLO'],
    ['7', 'Кузов', 'Кузов-и-система-отопления.22', 'FW POLO'],
    ['8', 'Жидкости', 'Топливо-и-эксплуатационные-жидкости.23', 'FW POLO'],
    ['9', 'Сервис', 'Гарантийное-и-сервисное-обслуживание.16', 'FW POLO'],
    ['10', 'Эксплуатация', 'Общие-вопросы-эксплуатации.17', 'FW POLO']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [16]:
def collect_themes(site, folder):
    
    site_address = site + 'forums/' + quote_plus(folder) + '/'
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for li in soup.find_all('li'):
            if li.find('h3'):
                h3 = li.find('h3')
                if h3.find('a'):
                    link = h3.find('a')
                    s = link.get('href')
                    theme = site + s
                    themes.append([link.get_text(), theme])
        
        for link in soup.find_all('a'):
            if link.has_attr('href'):
                if link.has_attr('class') and link.get('class') == ['text']:
                    if re.search('Вперёд', link.get_text()):
                        flag = True
                        s = link.get('href')
                        site_address = site + s 
                        break
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [17]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        flag = True
        theme_messages = []
        post_data = {}

        while flag:
            try:
                resp = urlopen(theme_address) # скачиваем файл
            except:
                break
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            lis = soup.find_all('li')
            for li in lis:
                if li.has_attr('id') and re.search('post-', li['id']):
                    author = li['data-author']
                    for div in li.find_all('div'):
                        if div.has_attr('class') and div['class'] == ['messageContent']:
                            blockquote = div.find('article').find('blockquote')
                            if blockquote.find('div'):
                                quote = blockquote.find('div')
                                quote.clear()
                            s = blockquote.get_text().strip()
                            s = re.sub('[\']+', '`', s)
                            s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                            post_date = ''
                            more_divs = div.find_next_siblings('div')
                            for next_div in more_divs:
                                for span in next_div.find_all('span'):
                                    if re.search('[0-9]+ [а-я]+ [0-9]{4}', span.get_text().strip()):
                                        post_date = re.search('[0-9]+ [а-я]+ [0-9]{4}', span.get_text().strip())[0]
                                        break
                            if len(s) > 2:
                                theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if link.has_attr('href'):
                    if link.has_attr('class') and link.get('class') == ['text']:
                        if re.search('Вперёд', link.get_text()):
                            flag = True
                            s = link.get('href')
                            theme_address = site + s
                            break
        if len(theme_messages) > 0:
            messages.append(['\''+ theme[0] + '\'', theme_messages])
        
    return messages

In [27]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [19]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [29]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [30]:
%%telegram_send Next message
scraper(site, folder_names)
'information collected'

Проблемы, заводской брак и тд.
Пробег от 100 000 (до 200 000) км
Пробег от 200 000 км...
Пробег от 0 до 15.000 км.
Пробег от 50 000км (до 100 000км)
(РЕШЕНО) Прогрев салона
Не в восторге...
Живые фотографии Поло седан ( Обсуждение фотографий ) [Как выкладывать фотки на форум]
Пробег от 15 000км (до 50 000км)
Тест-Драйв поло седан
Хочу купить авто но есть вопрос по поводу АКПП (Тема будет перенесена).
Коробка Автомат, обычный .
Купил новый поло 2018 Буду рад любым советам
Поло после Октавии; или Дауншифтинг, вот ты какой!
Забрызгивает двигатель CWVA
Свежий поло седан после старого меган 3
Книга жалоб и благодарностей, обсуждаем Официальных дилеров (+ опрос)
Продал свой Polo спустя 8 лет
Новый кузов Mk6 и PQ25vsPQ26
Пробег на момент покупки
Гул
Явно Будет отрицательный отзыв)
Положительные стороны Volkswagen Polo Sedan
Всем привет подскажи те пожалуйста может после замены воздушного фильтра повыситься расход топлива
Вопрос по комплектации
Продал Хонда Сивик, купил Поло Седан
Поло после П

Чип-тюнинг Поло Седан. (+ опрос)
Гул под капотом!!!
Не работает бензонасос.
Двигатель и его составляющие
(РЕШЕНО) Характерный щелчек в районе двигателя
Постукивания в районе двигателя стали громче и продолжительнее
Самая полная информация о двигателе Polo sedan.
Холостые обороты на теплом моторе (тема будет перенесена)
Сигнальная лампа давления масла.
(Тема будет перенесена) Удаление воздуха из системы охлаждения нашего ДВС
Все о двигателе CZCA 1.4 TSI 125л.с.
Работа гидрокомпенсаторов и замена поршневой группы (+ опрос)
Троит двигатель
Ремень ГРМ замена, время цена, и тд
Задолбал меня впускной коллектор
НЕТ ДАВЛЕНИЯ НА ПРОГРЕТОМ ДВИГАТЕЛЕ CFNA
Подсос воздуха через прокладку впускного коллектора
Проблемы с печкой
Датчик температуры двигателя своими руками
Для тех, у кого стучит двигатель
Новый мотор 1.6 CVWA поколения ЕА211 - особенности- масложор и пр.
Уплывает топливо без ошибок
отверстие в верхней головке шатуна cfna
Горит индикатор давления масла
Постель распредвалов
Удаление катал

После замены масла мигает лампа масла на горячем движке
Не заводится после простоя на морозе
Светодиод контроля min давления (0,5бар) масла ДВС
Подушка АКПП(левая)
Не заводится..
Прогрев двигателя
Запуск холодного двигателя и прогрев
После ремонта двигателя вибрирует и на компьютере дает ошибку
После ремонта двига вибрирует и звук похоже стал как дизельный
пропадает антифриз
Катализатор и его замена на пламегаситель в поло седан
Правильный уровень масла на щупе
Бензонасос снятие и чистка
Можно ли заменить цепь грм без специнструмента
Антифриз пониже отметки Минимум
ПРОБЛЕМА ПРОБЛЕМА
Вибрация двигателя при разгоне.
Правильная установка задающего кольца ДПКВ  1.6
Замена подушки двигателя
Дергается прогретый двигатель!
Кипит ОЖ
помогите свист в моторном отсеки при нагреве двигателя?
(РЕШЕНО) Замена ремня генератора и роликов
(РЕШЕНО) Джеттер (Jetter) приносит ли пользу?
Плохо заводится по утрам
Свечи NGK №44
Холостой ход
(Предварительно РЕШЕНО) Замена охлаждающей жидкости
Отсоединился воз

про АКПП (автоматическая коробка передач). Проблемы, вопросы, ошибки...
Когда стоит менять масло в МКПП?
Проблемы переключения с 1 на 2ю(МКПП)
Ошибка АКПП, аварийный режим, нужна помощь !
Масло в АКПП
шаровая опора вилки выключения сцепления
Пинки в АКПП при наборе скорости и включении задней передачи.
Подвывание Акпп
Рывки/толчки при торможении и трогании АКПП
Ресурс сцепления
(РЕШЕНО) Менять ли масло в АКПП?
Чип-тюнинг DSG 7
Проблемное включение 1 ой передачи
Замена механической коробки передач на "шестиступку"
Не втыкается задняя
Замена минералки на синтетику
Проблема с переключением
На холодную идет звук при не выжатом сцеплении.
Мкпп 2012 гв 271600
АКПП переключение
Проблема с переключением передач.
Металлический стук при переключении с 3-ей на 4-ую
Хруст при включение задней передачи
Коробка передач
Не фиксируется шланг от бачка на штуцере главного цилиндра сцепления
При нажатии на газ скорость растёт, обороты нет
Помогите разобраться в работе АКПП
Dsg7 1.4 tsi
Замена масла
Регул

Шины и колесные диски
Стойки стабилизатора
Проблемы с Рулём.. мб ЭУР.
Поломка ABS из-за разной размерности резины?
(РЕШЕНО) "Чиркает" спереди слева
Бряканье сзади в правом заднем колесе
Передние тормоза (передние дисковые тормоза и колодки), замена и эксплуатация
Замена амортизаторов
Вибрация автомобиля!
Внутренний шрус (трипоиды)
ШРУС
Поворотный кулак
Передняя подвеска после ДТП.
Нужен совет по передним тормозам
Замена задних колодок (барабан)
Тормозная система
Установка задних пружин. Проблемы и решения
Стук в передней подвеске
Втулки стабилизатора передние.
Клинит задние тормоза в плюсовую температуру
Глухой стук в ходовой
тормозная система
Болтающийся стук спереди
Передний амортизаторы
Задние барабанные колодки
Шины и колесные диски
Стук подвески при переезде лежачего полицейского
Стуки при повороте руля
Шаровые опоры.
Сайленблоки передние
колодки на задние дисковые тормоза
Опоры передние неисправности
ЭУР зимой работает, в летом нет. Поло 2002
Датчик ABS
Несимметричное положение у

...

Замена масла в ДВС (CFNA) Поло Седан (+ опрос)
«iMagnet»,SUPROTEC-личный опыт
Какой бензин заливать: АИ92, АИ95, АИ98 ? (+ опрос)
(РЕШЕНО) Промывка перед заменой масла другой марки?
Залив масла в ДВС.
Антифриз.
Замена тормозной жидкости и прокачка тормозов
Тормозная жидкость и её замена
Установка газового оборудования
Болт сливного отверстия поддона .
(РЕШЕНО) Проверка уровня моторного масла
Интересует реальный расход топлива(только владельцы)
Омывающая жидкость, выбор омывайки, как разморозить, слить воду в бачке омывающей жидкости
Метан он же Ecogas, он же Ecofuel и всё, что с ним связано
Моторное масло ROWE
Расход топлива после снятия АКБ
Замена масла после обкатки двигателя
(РЕШЕНО) Слив бензина из бензобака.
(РЕШЕНО) Деления бака
Масло Супротек как в использовании
Лобовое в маленьких сколах.
Посоветуйте, как достать стакан бензонасоса (фото)
Капельный анализ масла ДВС
Где и сколько стоит топливо?
Гул в районе топливного насоса и выбор фильтра.
G 001 770 A2
Расход масла
Двигатель г

Техническое Обслуживание (ТО) VW Polo Sedan: обсуждение, стоимость
Второе письмо в  Volkswagen Group Rus
Какие слабые места у нашего поло? (статистика поломок)
Что важно знать, когда автомобиль "снимают с гарантии"!
Сервисная акция для владельцев VW Polo седан №4 (по замене  каркасов сидений) код 72Е4/01
Расходные материалы для VW POLO седан
Сервисная акция для владельцев VW Polo седан №5 (по защите проводов АБС) код 92Z4/01
Сервисная акция для владельцев VW Polo седан  №1 (по перепрошивке ЭУР, С 21 апреля)
Сервисная акция для владельцев VW Polo седан №2 (по замене рулевых наконечников)
Сервисная акция для владельцев VW Polo седан  №3 (по замене замка багажника)
(РЕШЕНО) Как самому сбросить ключ ТО (INSP service)
Топливный фильтр и проблемы замены
Замена ветрового стекла из-за проблем с обогревом на исходе гарантии
Гарантия на Поло Седан
Нормальные автосервисы в Санкт-Петербурге
Изменение интервала сервисного обслуживания
Тахограф
Пропуски зажигания
Опрос: через сколько меняете масло в

Номера запчастей, оригинал и не оригинал! (Поиск деталей по номеру)
Подготовка к зиме, советы (Зимняя эксплуатация автомобиля)
Проблемы зимнего запуска
Проезд перекрёстков (Тема будет перенесена в "водительские премудрости")
Замена масла в двигателе
Травит воздух при отпускания педали тормоза при морозе от -10  и ниже на поло
Стук где то под торпедо или около ручки мкпп
Украли колпаки
Вибрация при низкой скорости на второй передаче и выше
Abs
Свист в машине
Затирается лобовое стекло.
Штатный парктроник не выводит вызуализацию на экран R140
Электропривод складывания зеркал?
Рывки на разгоне. Мкпп.
Лобовое стекло с подогревом...
Аэродинамический шум при движении
Замена стоейк от другой машины
значек снежинка на инфрмационнй понели фольсваген роло седан
Как читать схемы Elcats для поиска деталей
ни с того, ни чего заглох двигатель при движении
Расход бензина на автомате.
Шум при трогании утром
Заржавел ключ замка зажигания
Внутренняя ручка передней двери (левая)
Шумоизоляция
Фары
Вой при 

Как снять бачек омывателя лобового стекла?
БЕЗОПАСНОСТЬ ПРИ ВОЖДЕНИИ!
(РЕШЕНО) Как добраться до  свечей
Прокачай Поло Седан!!!!
Греется двигатель

Information collected


'information collected'

In [ ]:
import pandas as pd
mess = pd.read_csv('FW POLO_14_Проблемы-Заводской брак.csv', quotechar="'")
mess

In [ ]:
site = 'http://www.polosedan-club.com/'
folder_names = [
    ['0', 'Отзывы', 'Отзывы-владельцев-о-Поло-седан-polo-sedan.13', 'FW POLO']
]
for folder_name in folder_names:
    if len(folder_name[2]) > 0:
        folder = folder_name[2]
        current_themes = collect_themes(site, folder)

In [ ]:
current_themes

In [ ]:
site = 'http://www.polosedan-club.com/'
#current_themes = [['Скрипы в торпеде(УДАЛИТЬ ПОСЛЕ ПРОЧТЕНИЯ ПОСТА МОДЕРАТОРА)',
#  'http://www.polosedan-club.com/threads/%D0%A1%D0%BA%D1%80%D0%B8%D0%BF%D1%8B-%D0%B2-%D1%82%D0%BE%D1%80%D0%BF%D0%B5%D0%B4%D0%B5-%D0%A3%D0%94%D0%90%D0%9B%D0%98%D0%A2%D0%AC-%D0%9F%D0%9E%D0%A1%D0%9B%D0%95-%D0%9F%D0%A0%D0%9E%D0%A7%D0%A2%D0%95%D0%9D%D0%98%D0%AF-%D0%9F%D0%9E%D0%A1%D0%A2%D0%90-%D0%9C%D0%9E%D0%94%D0%95%D0%A0%D0%90%D0%A2%D0%9E%D0%A0%D0%90.7996/']]
current_messages = collect_messages(site, current_themes)

In [ ]:
current_messages

In [ ]:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])] = [author_id, author_nickname]
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        author = post_data[message_id][1]
                    else:
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            if item.has_attr('class') and item['class'][0] == 'smallfont':
                                post_date = item.get_text().strip()
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s) + '\''
